Predecir la cantidad de productos por vuelo
Importamos librerias y cargamos el primer dataset Sales, para verlo y limpiarlo

In [35]:
import pandas as pd
import matplotlib as plt
import numpy as np

df = pd.read_csv("Sales TEC_Valid.csv")

Eliminamos los valores de ProductType que no nos importen

In [36]:
types_to_include = [
    "Botanas",
    "Refrescos",
    "Perecederos",
    "Galletas",
    "Bebidas Calientes",
    "Sopas",
    "Lacteos",
    "Alimentos Charter"
]

df = df[df['ProductType'].isin(types_to_include)]

Transformamos el df para que solo aparezca una vez cada vuelo, dejamos el id del vuelo y a cada vuelo le añadimos los articulos que se consumen.

In [37]:
df = df.pivot_table(index='Flight_ID', columns='ProductName', values='Quantity', fill_value=0)

# Resetear el índice para que Flight_ID sea una columna nuevamente
df.reset_index(inplace=True)
df

ProductName,Flight_ID,Agua Natural 600 Ml,Arandano,Arandano Mango Mix,Arcoiris,Cafe 19 Cafe Clasico,Cafe 19 Capuchino,Cafe 19 Chiapas,Cafe Costa,Cafe De Olla,...,Sabritas Originales,Salsa Botanera,Sidral Mundet,Sprite,Te Frutos Rojos,Te Manzanilla Jengibre,Te Relax,Te Vainilla,Tostitos,Tostitos Nachos Con Dip
0,00004a718edba9d9ef878d08f02ae057,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0000cd79c0c3a9c309df6064dcacaeea,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,000163f0df9cbfc35c4c06645ec512f6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00017be73003a570dd426b155762769c,8.0,0.0,1.0,0.0,0.0,2.0,0.0,4.0,0.0,...,12.0,0.0,2.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0001a43836c338f8d8650aefb11672c9,3.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,...,3.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108448,fffe47f84ecc55da94b2907a7317dd12,15.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,0.0
108449,fffe4a9c19ae2320e76f02939fdff957,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,4.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0
108450,ffff138536f249f69340e0a8336f94a4,5.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
108451,ffff782f2ae79e385a016c00fbd994b8,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


Cargamos los datos de Flights, pero ahora ya limpios y listos para añadirlos a nuestro df

In [38]:
df2 = pd.read_csv('datosvuelosfinal.csv')

# Realizar un inner join
combined_df = pd.merge(df, df2, on='Flight_ID', how='inner')
combined_df = combined_df.drop(combined_df.columns[74], axis=1)

# Guardar el DataFrame combinado en un nuevo archivo CSV
#combined_df.to_csv('dataset_combinado.csv', index=False)

In [39]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104526 entries, 0 to 104525
Data columns (total 99 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Flight_ID                               104526 non-null  object 
 1   Agua Natural 600 Ml                     104526 non-null  float64
 2   Arandano                                104526 non-null  float64
 3   Arandano Mango Mix                      104526 non-null  float64
 4   Arcoiris                                104526 non-null  float64
 5   Cafe 19 Cafe Clasico                    104526 non-null  float64
 6   Cafe 19 Capuchino                       104526 non-null  float64
 7   Cafe 19 Chiapas                         104526 non-null  float64
 8   Cafe Costa                              104526 non-null  float64
 9   Cafe De Olla                            104526 non-null  float64
 10  Carne Seca Habanero                     1045

Se escalan los datos, pero solo los datos de entrada. De esta manera podemos tener una mejor manera para entrenar con datos pequeños.

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

'''output_columns = [col for col in combined_df.columns if 'Agua Natural 600 Ml' <= col <= 'Tostitos Nachos Con Dip']
input_columns = combined_df.columns  # Excluye las primeras dos columnas
input_columns = [col for col in input_columns if col not in output_columns]
'''

X = combined_df.iloc[:,74:]
y = combined_df.iloc[:, 1:74]


# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalización de características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Contar ceros en todas las columnas

In [41]:
# Contar ceros en todas las columnas
#pd.set_option('display.max_rows', None)  # O establecer un número específico mayor


zeros_count_per_column = y.eq(0).sum()
print("Número de ceros por columna:")
#zeros_count_per_column
zeros_count_per_column.sort_values(ascending=False)

Número de ceros por columna:


Kacang Flaming Hot           104525
Gomita Enchilada La Cueva    104525
Club Sandwich                104525
Nissin Limon Y Habanero      104523
Cuerno Individual Charter    104523
                              ...  
Doritos Nacho                 34506
Ruffles Queso                 34194
Sabritas Originales           24740
Coca Cola Regular             11094
Agua Natural 600 Ml           10616
Length: 73, dtype: int64

In [42]:
X_train_scaled = np.array(X_train_scaled).astype('float32')
X_test_scaled = np.array(X_test_scaled).astype('float32')

# Convierte las etiquetas de salida a float32
y_train = np.array(y_train).astype('float32')
y_test = np.array(y_test).astype('float32')

In [43]:
#pip install tensorflow


Creamos un modelo secuencial de tres layers, las tres de manera no lineal. El layer de salida es del tamaño de todos los datos a predecir, que son los artivulos por cada vuelo.

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
# Construcción del modelo
# Construcción del modelo

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(y_train.shape[1], activation='linear')  # Uso de activación lineal para regresión
])

# Compilación del modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])


/opt/anaconda3/envs/tecmx/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [45]:
# Entrenamiento del modelo
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32,validation_split=0.1)

# Evaluación del modelo
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')


Epoch 1/50
2352/2352 ━━━━━━━━━━━━━━━━━━━━ 2s 481us/step - loss: 1.6720 - mae: 0.5783 - val_loss: 1.4307 - val_mae: 0.5298
Epoch 2/50
2352/2352 ━━━━━━━━━━━━━━━━━━━━ 1s 433us/step - loss: 1.3746 - mae: 0.5292 - val_loss: 1.3766 - val_mae: 0.5388
Epoch 3/50
2352/2352 ━━━━━━━━━━━━━━━━━━━━ 1s 442us/step - loss: 1.3124 - mae: 0.5170 - val_loss: 1.3422 - val_mae: 0.5090
Epoch 4/50
2352/2352 ━━━━━━━━━━━━━━━━━━━━ 1s 422us/step - loss: 1.2940 - mae: 0.5132 - val_loss: 1.3388 - val_mae: 0.5072
Epoch 5/50
2352/2352 ━━━━━━━━━━━━━━━━━━━━ 1s 418us/step - loss: 1.2830 - mae: 0.5093 - val_loss: 1.3267 - val_mae: 0.5275
Epoch 6/50
2352/2352 ━━━━━━━━━━━━━━━━━━━━ 1s 413us/step - loss: 1.2689 - mae: 0.5079 - val_loss: 1.2927 - val_mae: 0.5059
Epoch 7/50
2352/2352 ━━━━━━━━━━━━━━━━━━━━ 1s 413us/step - loss: 1.2525 - mae: 0.5051 - val_loss: 1.3314 - val_mae: 0.5058
Epoch 8/50
2352/2352 ━━━━━━━━━━━━━━━━━━━━ 1s 412us/step - loss: 1.2529 - mae: 0.5062 - val_loss: 1.3041 - val_mae: 0.5105
Epoch 9/50
2352/2352 ━━━

In [46]:
from sklearn.metrics import mean_squared_error, r2_score


scaler_X = StandardScaler()  # O MinMaxScaler()
scaler_Y = StandardScaler()  # O MinMaxScaler()
# Predicción y desescalado para obtener resultados en la escala original

y_pred_scaled = model.predict(X_test_scaled)
#y_pred = scaler_Y.inverse_transform(y_pred_scaled)

# Métricas adicionales
mse = mean_squared_error(y_test, y_pred_scaled)
r2 = r2_score(y_test, y_pred_scaled)

print(f'Mean Squared Error: {mse}, R2 Score: {r2}')

654/654 ━━━━━━━━━━━━━━━━━━━━ 0s 300us/step
Mean Squared Error: 1.2328035831451416, R2 Score: 0.15621340326013042


El valor de MSE queda pequeño, sin embargo en el contexto del problema resulta ser muy grande, ya que la mayoria de articulos no son vendidos.Esto causa que el R2 score sea muy pequeño, ya que como la mayoria son ceros, que sse caluclo aunque sea uno, causa mucha diferencia. Sin embargo, consideramos que es es el mejor modelo que podemos conseguir.

Des escalamos los datos.

In [47]:
y_pred_scaled_round= np.round(y_pred_scaled).astype(int)
y_pred_scaled_round


array([[4, 0, 0, ..., 0, 1, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       ...,
       [6, 0, 0, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 1, 0],
       [2, 0, 0, ..., 0, 0, 0]])

In [48]:
df3 = pd.read_csv('dataset2024_pass_predicted.csv')

In [49]:
df3_x=df3.drop(columns=['Flight_ID'])

In [50]:
df3_x1=df3_x.drop(columns=['Capacity'])
df3_x1.reset_index(drop=True, inplace=True)


In [51]:
new_column_order = ['Expected_Passengers', 'Expected_Bookings'] + [col for col in df3_x1.columns if col not in ['Expected_Passengers', 'Expected_Bookings']]

df3_x1 = df3_x1[new_column_order]

Hacemos la predicción para los vuelos de 2024

In [52]:

#combined_df = pd.merge(df, df2, on='Flight_ID', how='inner')
#combined_df = combined_df.drop(combined_df.columns[74], axis=1)
df3_x1.rename(columns={'Expected_Passengers': 'Passengers', 'Expected_Bookings': 'Bookings'}, inplace=True)
df3_x1 = scaler.fit_transform(df3_x1)
df3_x1 = np.array(df3_x1).astype('float32')

y_pred_2024 = model.predict(df3_x1)



54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [53]:
y_pred_2024= np.round(y_pred_2024).astype(int)
y_pred_2024

array([[2, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       ...,
       [6, 0, 0, ..., 0, 1, 0],
       [5, 0, 0, ..., 0, 1, 0],
       [6, 0, 0, ..., 0, 1, 0]])

In [54]:
new_column_names = [
    "Agua Natural 600 Ml", "Arandano", "Arandano Mango Mix", "Arcoiris",
    "Cafe 19 Cafe Clasico", "Cafe 19 Capuchino", "Cafe 19 Chiapas", "Cafe Costa",
    "Cafe De Olla", "Carne Seca Habanero", "Carne Seca Original", "Cheetos",
    "Cheetos Flamin Hot", "Chokis", "Ciel Mineralizada", "Club Sandwich",
    "Coca Cola Dieta", "Coca Cola Regular", "Coca Sin Azucar", "Cuerno Clasico De Pavo",
    "Cuerno Individual Charter", "Dip De Queso", "Doritos Nacho", "Eco Holder",
    "Emperador Chocolate", "Emperador Vainilla", "Fanta De Naranja", "Fritos Limon Y Sal",
    "Frutos Secos Enchilados", "Galleta De Arandano Relleno De Q/Crema",
    "Galleta De Chispas De Chocolate", "Galleta De Chocolate", "Go Nuts",
    "Gomita Enchilada La Cueva", "Jugo De Mango", "Jugo De Manzana", "Kacang Flaming Hot",
    "Leche De Chocolate Sc", "Leche De Fresa Sc", "Luxury Nut Mix", "Mafer Sin Sal",
    "Maxi Combo", "Mega Cuerno Clasico", "Muffin Integral", "Nishikawa Japones",
    "Nishikawa Salado", "Nissin Dark Dragon", "Nissin Fuego", "Nissin Limon Y Habanero",
    "Nissin Picante", "Nissin Res", "Nueces De Arbol Mix", "Nutty Berry Mix",
    "Panini Clasico", "Panini Integral", "Protein Adventure", "Quaker Avena Frutos Rojos",
    "Quaker Avena Moras", "Quaker Granola", "Quaker Natural Balance", "Rancheritos",
    "Ruffles Queso", "Sabritas Flamin Hot", "Sabritas Originales", "Salsa Botanera",
    "Sidral Mundet", "Sprite", "Te Frutos Rojos", "Te Manzanilla Jengibre", "Te Relax",
    "Te Vainilla", "Tostitos", "Tostitos Nachos Con Dip"
]

df_new_columns = pd.DataFrame(y_pred_2024, columns=new_column_names)

# Añadir las nuevas columnas al costado derecho del DataFrame original
df_final = pd.concat([df3.reset_index(drop=True), df_new_columns], axis=1)
df_final

,Flight_ID,Capacity,tiempoVuelo,Destination_Type_Ciudad Fronteriza,Destination_Type_Ciudad Principal,Destination_Type_Ecoturismo,Destination_Type_MX Amigos y Familia,Destination_Type_Playa,Origin_Type_Ciudad Fronteriza,Origin_Type_Ciudad Principal,...,Sabritas Originales,Salsa Botanera,Sidral Mundet,Sprite,Te Frutos Rojos,Te Manzanilla Jengibre,Te Relax,Te Vainilla,Tostitos,Tostitos Nachos Con Dip
0,c8e2539e6b04432617d945b30bdd9d69,180,75.0,False,False,False,False,True,False,True,...,1,0,0,0,0,0,0,0,0,0
1,5717f9937ebe931d3b285ae4c193bb6b,180,45.0,False,False,False,False,True,False,True,...,3,0,1,1,0,0,0,0,0,0
2,dbddb026c6d57cd24ea29936ea5cd171,180,75.0,False,False,False,False,True,False,True,...,2,0,0,1,0,0,0,0,0,0
3,cfae882b6e5a25c979a7455989db993e,186,100.0,True,False,False,False,False,False,True,...,2,0,0,1,0,0,0,0,0,0
4,c6eb44f49d835a76c865c75d1026aa25,180,50.0,False,False,False,True,False,False,True,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,764f5b604b85a8fd0e88a941139aefc2,180,200.0,False,False,False,False,True,False,True,...,3,0,1,1,0,0,0,0,0,0
1710,165c65a0cc06f7bd9cc4515a2b7eedb7,240,180.0,False,False,False,False,True,False,True,...,5,0,1,2,0,0,0,0,1,0
1711,629413c278109e02284933edba9cff60,240,180.0,False,False,False,False,True,False,True,...,5,0,1,2,0,0,0,0,1,0
1712,641573d2c38d4990397c7eb95100cd7a,240,180.0,False,False,False,False,True,False,True,...,5,0,1,2,0,0,0,0,1,0


In [55]:
df_final.to_csv("PrediccionFinal.csv", index=False)